In [1]:
print("문제 4-1 : 카페 메뉴 도구(Tool) 호출 체인 구현 ( LangChain 사용)")

문제 4-1 : 카페 메뉴 도구(Tool) 호출 체인 구현 ( LangChain 사용)


In [36]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
print(OPENAI_API_KEY[:2])
print(UPSTAGE_API_KEY[30:])
print(TAVILY_API_KEY[:2])

gs
3g
tv


In [40]:
from langchain_upstage import ChatUpstage
llm = ChatUpstage(
        model="solar-pro",
        base_url="https://api.upstage.ai/v1",
        temperature=0.5
    )
print(llm)

# from langchain_openai import ChatOpenAI
# llm = ChatOpenAI(
#     model='gpt-3.5-turbo-0125', 
#     temperature=0,
# )
# print(llm.model_name)

client=<openai.resources.chat.completions.completions.Completions object at 0x0000025E0C085A90> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000025E0C0868A0> model_name='solar-pro' temperature=0.5 model_kwargs={} upstage_api_key=SecretStr('**********') upstage_api_base='https://api.upstage.ai/v1'


In [44]:
from langchain_community.document_loaders import TextLoader
from langchain_upstage import UpstageEmbeddings

from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS

# 데이터 로드 및 분할

loader = TextLoader("./data/cafe_menu.txt", encoding="utf-8")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

# 벡터 DB 생성 및 저장

embeddings = UpstageEmbeddings(model="solar-embedding-1-large")

db = FAISS.from_documents(docs, embeddings)

db.save_local("./db/cafe_db_upstage")

print("벡터 DB가 './db/cafe_db_upstage' 폴더에 성공적으로 생성 및 저장되었습니다.")

벡터 DB가 './db/cafe_db_upstage' 폴더에 성공적으로 생성 및 저장되었습니다.


In [ ]:
from langchain_core.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_upstage import ChatUpstage 

# Tavily 웹 검색 도구
tavily_search_func = TavilySearchResults(max_results=1, name="tavily_search_tool")

# 위키피디아 검색 및 요약 도구
@tool
def wiki_summary(query: str) -> str:
    """
    Searches Wikipedia for the given query and returns a summary of the top result.
    Use this tool for general knowledge questions about topics, people, concepts, etc.
    """
    wiki_client = WikipediaAPIWrapper()
    return wiki_client.run(query)

# 로컬 카페 메뉴 DB 검색 도구 
@tool
def db_search_cafe_func(query: str) -> list:
    """
    Searches the local cafe menu vector database for information about menu items,
    including their price, ingredients, and description. Use this for any questions
    related to the cafe's menu.
    """
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    db = FAISS.load_local("./db/cafe_db_upstage", embeddings, allow_dangerous_deserialization=True)
    retriever = db.as_retriever(search_kwargs={"k": 2})
    
    result_docs = retriever.invoke(query)
    return [doc.page_content for doc in result_docs]

tools = [tavily_search_func, wiki_summary, db_search_cafe_func]
llm = ChatUpstage(model="solar-1-mini-chat", temperature=0) 
llm_with_tools = llm.bind_tools(tools)


In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

@chain
def tool_chain(user_input: str):
    ai_msg: AIMessage = llm_with_tools.invoke([HumanMessage(content=user_input)])

    if not ai_msg.tool_calls:
        return ai_msg.content

    tool_msgs = []
    for call in ai_msg.tool_calls:
        tool_name = call["name"]
        args = call["args"]
        for t in tools:
            if t.name == tool_name:
                result = t.invoke(args["query"])
                tool_msgs.append(
                    ToolMessage(content=str(result), tool_call_id=call["id"])
                )

    final_response = llm_with_tools.invoke(
        [HumanMessage(content=user_input), ai_msg, *tool_msgs]
    )
    return final_response.content


In [ ]:
# 테스트 질문 실행
question = "아메리카노의 가격과 특징은 무엇인가요?"

response = tool_chain.invoke(question)

print(f"질문: {question}")
print(f"답변: {response}")

질문: 아메리카노의 가격과 특징은 무엇인가요?
답변: 아메리카노의 가격은 ₩4,500이며, 주요 원료는 에스프레소와 뜨거운 물입니다. 아메리카노는 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피로, 원두 본연의 맛을 가장 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽 추가가 가능합니다. 또한, 아이스 아메리카노도 있으며, 가격은 ₩4,500입니다. 아이스 아메리카노는 진한 에스프레소에 차가운 물과 얼음을 넣어 만든 시원한 아이스 커피로, 깔끔하고 시원한 맛이 특징이며, 더운 날씨에 인기가 높습니다.
